# **Rock-Paper-Scissors Classification with Convolutional Neural Networks (CNN)**

## Overview

In this notebook, we will build a Convolutional Neural Network (CNN) to classify images of hand gestures: Rock, Paper, and Scissors. The process will include:

1. **Importing Libraries**
2. **Data Loading and Preparation**
3. **Model Building**
4. **Model Training**
5. **Model Evaluation**
6. **Making Predictions on New Images**

We will use TensorFlow and Keras for building the model and work through each step with detailed explanations.

### **1. Importing Necessary Libraries**

We start by importing essential libraries:

- **TensorFlow**: Main library for building and training the CNN.
- **TensorFlow Datasets**: For loading the Rock-Paper-Scissors dataset.
- **NumPy**: For numerical operations.
- **Matplotlib.pyplot**: For data visualization.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import os

### **2. Loading and Preparing the Dataset**

We will load the Rock-Paper-Scissors dataset from TensorFlow Datasets and preprocess it by:

- **Resizing Images**: Standardizing image sizes.
- **Normalizing Pixel Values**: Scaling pixel values to the range [0, 1].
- **Data Augmentation**: Enhancing the dataset with transformed images to improve model generalization.

In [ ]:
# Load the dataset
(train_ds, test_ds), ds_info = tfds.load(
    'rock_paper_scissors',
    split=['train', 'test'],
    as_supervised=True,
    with_info=True
)

# Define image size and batch size
IMG_SIZE = (150, 150)
BATCH_SIZE = 32

# Preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0  # Normalize pixel values
    return image, label

# Apply preprocessing to the datasets
train_ds = train_ds.map(preprocess).batch(BATCH_SIZE)
test_ds = test_ds.map(preprocess).batch(BATCH_SIZE)

### **Data Visualization**

Let's visualize a few samples from the training dataset to understand what the images look like.

In [ ]:
class_names = ds_info.features['label'].names

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.title(class_names[labels[i]])
        plt.axis('off')

### **4. Building the CNN Model**

We will build a Sequential CNN model consisting of:

- **Convolutional Layers**: To extract features from images.
- **Pooling Layers**: To reduce spatial dimensions.
- **Flatten Layer**: To convert the 2D feature maps into 1D feature vectors.
- **Dense Layers**: For classification based on extracted features.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=IMG_SIZE + (3,)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 classes: Rock, Paper, Scissors
])

### **Compiling the Model**

We compile the model using:

- **Loss Function**: `sparse_categorical_crossentropy` for multi-class classification.
- **Optimizer**: `adam` optimizer for efficient training.
- **Metrics**: `accuracy` to evaluate the performance.

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

### **6. Training the Model**

We train the model using the training dataset for a certain number of epochs.

- **Epochs**: Number of times the model will cycle through the training data.

In [ ]:
EPOCHS = 10

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS
)

### **7. Evaluating the Model**

After training, we evaluate the model on the test dataset to check its performance on unseen data.

In [ ]:
test_loss, test_accuracy = model.evaluate(test_ds)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

### **Visualizing Training Results**

We plot the training and validation accuracy and loss to visualize the model's performance over epochs.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy over Epochs')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Loss over Epochs')
plt.show()

### **9. Making Predictions on New Images**

We can use the trained model to predict the class of new images.

- **Load and Preprocess the Image**: Resize and normalize the image.
- **Predict**: Use `model.predict` to get the prediction probabilities.
- **Interpret the Result**: Find the class with the highest probability.

In [ ]:
def predict_image(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=IMG_SIZE)
    image_array = tf.keras.preprocessing.image.img_to_array(image) / 255.0
    image_array = tf.expand_dims(image_array, 0)  # Create batch axis

    predictions = model.predict(image_array)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = np.max(predictions[0])

    plt.imshow(image_array[0])
    plt.title(f'Prediction: {predicted_class} ({confidence * 100:.2f}%)')
    plt.axis('off')
    plt.show()

### **Testing the Model with a New Image**

Let's test the model with a new image of a hand gesture.

In [ ]:
# Provide the path to your image
image_path = 'scissor.jpg'
predict_image(image_path)